# Implementing the Transformer

Reference: [Implementation_Tutorial](Transformer_Implementation_Tutorial.ipynb)

In [1]:
from torch import nn 
import torch.nn.functional as F
import torch
from math import log, sqrt

In [2]:
dev = 'mps' if torch.backends.mps.is_available() else 'cpu'

## Embdedding and Position Encoding Module

In [3]:
class EmbeddingWithPositionalEncoding(nn.Module):
    def __init__(self, vocab_size: int, 
                 d_embed: int, 
                 d_model: int,
                 dropout_p: float = 0.1
                 ):
        super().__init__()
        self.d_model = d_model
        self.d_embed = d_embed
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=d_embed,
            device=dev
        )
        self.projection = nn.Linear(
            in_features=d_embed,
            out_features=d_model,
            device=dev
        )
        self.scaling = float(sqrt(self.d_model))

        self.layerNorm = nn.LayerNorm(
            self.d_model,
            device=dev
        )
        
        self.dropout = nn.Dropout(p=dropout_p)

    @staticmethod # decorator that indicates that the following function doesn't operate on `self`
    def create_positional_encoding(seq_length:int, 
                                   d_model:int, 
                                   batch_size:int
                                   ):

        positions = torch.arange(seq_length, dtype=torch.long, device=dev)\
            .unsqueeze(1) # shape (seq_length, 1) i.e. makes it vertical
        
        div_term = torch.exp(
            (torch.arange(0, d_model, 2)/d_model)*(-4)*log(10)
        ).to(dev)
        
        pe = torch.zeros(size=(seq_length, d_model), dtype=torch.long, device=dev) # the tensor to be multiplied to positions tensor to get pe
        pe[:, 0::2] = torch.sin(positions*div_term) # for even dimensions
        pe[:, 1::2] = torch.cos(positions*div_term) # for odd dimensions
        pe = pe.unsqueeze(0).expand(batch_size, -1, -1) # copy out the encodings for each batch
        return pe
    
    def forward(self, x):
        batch_size, seq_length = x.shape

        # step 1: make embeddings
        token_embedding = self.embedding(x)

        # step 2: go from d_embed to d_model
        token_embedding = self.projection(token_embedding) \
            * self.scaling # multiplying with scaling factor, just like in the paper

        # step 3: add positional encoding
        pos_encoding = self.create_positional_encoding(
            seq_length=seq_length, 
            d_model = self.d_model,
            batch_size=batch_size
        )

        #step 4: normalize the sum of pos encoding and token_embed
        norm_sum = self.layerNorm(pos_encoding + token_embedding)
        op = self.dropout(norm_sum)
        return op



## Attention Module

- Two types of attention I learnt:
  - **Self-Attention:** key values come from the same input tensor
  - **Cross-Attention:** key values come fromt he output of a different multi-head attention module

In [4]:
class TransformerAttention(nn.Module):
    def __init__(self, 
                 d_model: int,
                 num_heads: int,
                 dropout_p: float = 0.1,
                 dev='cpu'
                 ):
        super().__init__()
        if (d_model % num_heads) != 0: raise ValueError(f'`d_model` not divisible by `num_heads`')
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_heads = self.d_model // self.num_heads
        self.scale_factor = float(1.0 / sqrt(self.d_heads))
        self.dropout = nn.Dropout(p=dropout_p)
        self.dev = dev

        #linear transformations
        self.q_proj = nn.Linear(
            in_features=self.d_model,
            out_features=self.d_model,
            device=self.dev
        )

        self.k_proj = nn.Linear(
            in_features=self.d_model,
            out_features=self.d_model,
            device=dev
        )

        self.v_proj = nn.Linear(
            in_features=self.d_model,
            out_features=self.d_model,
            device=dev
        )

        self.output_proj = nn.Linear(
            in_features=self.d_model,
            out_features=self.d_model,
            device=dev
        )

    def forward(self, 
                seq: torch.Tensor, 
                key_value_states:torch.Tensor = None, 
                att_mask: torch.Tensor = None):
        batch_size, seq_length, d_model = seq.size()

        Q_state: torch.Tensor = self.q_proj(seq)
        if key_value_states is not None:
            kv_seq_len = key_value_states.size(1)
            K_state: torch.Tensor = self.k_proj(key_value_states)
            V_state: torch.Tensor = self.v_proj(key_value_states)
        else:
            kv_seq_len = seq_length
            K_state: torch.Tensor = self.k_proj(seq)
            V_state: torch.Tensor = self.v_proj(seq)

        Q_state = Q_state.view(batch_size, seq_length, self.num_heads, self.d_heads).transpose(1, 2)
        K_state = K_state.view(batch_size, kv_seq_len, self.num_heads, self.d_heads).transpose(1, 2)
        V_state = V_state.view(batch_size, kv_seq_len, self.num_heads, self.d_heads).transpose(1, 2)

        Q_state = Q_state * self.scale_factor
        
        self.att_matrix = torch.matmul(Q_state, K_state.transpose(-1, -2))
        

        if att_mask is not None:
            self.att_matrix = self.att_matrix + att_mask # yes, in this case the mask is not multiplied, but added. This is to ensure that after softmax the things to be excluded are 0
        
        att_score = F.softmax(self.att_matrix, dim=-1) # torch.nn.Softmax() is used in __init__, F.softmax() is used for these inline operations.
        att_score = self.dropout(att_score)
        att_op = torch.matmul(att_score, V_state)

        #concatenating all heads 
        att_op = att_op.transpose(1, 2)
        att_op = att_op.contiguous().view(batch_size, seq_length, self.num_heads*self.d_heads)

        att_op = self.output_proj(att_op)

        return att_op



In [5]:
enc = EmbeddingWithPositionalEncoding(
    vocab_size=100,
    d_embed=512, 
    d_model=256
)

att_layer = TransformerAttention(
    d_model=256, 
    num_heads=8
)

x = enc(torch.tensor([1, 2, 3], device=dev).unsqueeze(0))
att_layer(x)

tensor([[[ 2.2453e-01,  1.5356e-03,  9.8504e-02, -3.6989e-02, -2.0104e-01,
           3.0645e-02, -4.8723e-01, -2.8341e-01,  7.7438e-02,  2.9908e-01,
          -1.4741e-01,  1.4592e-01,  1.7704e-01,  1.1769e-01,  4.1054e-02,
          -2.1217e-01, -3.8033e-01,  1.4640e-01,  5.0318e-02,  5.0131e-01,
           5.9424e-01,  3.8934e-01, -2.9320e-02,  3.0623e-01, -1.8369e-01,
          -7.2738e-02,  2.0785e-01,  4.2893e-01,  3.0858e-01,  3.7215e-01,
          -6.4216e-01, -2.8656e-01,  4.6259e-01, -1.7676e-01,  3.0973e-01,
          -1.8818e-01,  8.9793e-02, -5.8858e-01, -8.1687e-02,  1.2802e-01,
          -7.9837e-02, -1.0723e-01,  7.6052e-02, -5.3435e-01,  2.9916e-01,
          -2.1117e-01, -6.2239e-01,  3.0656e-02,  1.9151e-01,  3.8035e-01,
           1.0984e-01,  2.0858e-01,  8.7536e-02,  3.2996e-01, -2.9303e-01,
          -1.1950e-01,  3.0866e-02,  2.6978e-01,  5.3533e-02,  4.4368e-01,
           2.6899e-01,  2.2969e-02, -1.1797e-01,  4.2570e-02, -2.7551e-01,
          -4.7569e-01, -5

## Feed-Forward Network

- According, to section 3.3 of the paper, this has 2 layers
- d_model -> d_ff -> d_model
- same parameters for every position.

In [6]:
class FeedForwardNetwork(nn.Module):
    def __init__(self,
                 d_model: int,
                 d_ff: int):
        
        super().__init__()

        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = nn.Linear(
            in_features=self.d_model,
            out_features=self.d_ff,
            device=dev
        )

        self.fc2 = nn.Linear(
            in_features=self.d_ff,
            out_features=self.d_model,
            device=dev
        )
        
    def forward(self, input:torch.Tensor):
        batch_size, seq_length, d_input = input.size()
        f1 = F.relu(self.fc1(input))
        f2 = self.fc2(f1)
        return f2


## The Encoder Module

In [7]:
class TransformerEncoder(nn.Module):
    def __init__(self,
                 d_model: int, 
                 num_heads: int,
                 d_ff: int,
                 dropout_p = 0.1
                 ):
        
        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.dropout_p = dropout_p
        self.d_ff = d_ff

        self.att_layer = TransformerAttention(
            d_model=self.d_model,
            num_heads=self.num_heads,
            dropout_p=self.dropout_p
        )

        self.ffn = FeedForwardNetwork(
            d_model=self.d_model,
            d_ff = self.d_ff
        )

        self.norm1 = nn.LayerNorm(self.d_model, device=dev)
        self.norm2 = nn.LayerNorm(self.d_model, device=dev)

        self.dropout = nn.Dropout(p=self.dropout_p)
        
    def forward(self, x: torch.Tensor):
        x_att = self.att_layer(x)

        x_att = self.dropout(x_att)
        x_norm1 = self.norm1(x + x_att)

        x_ff = self.ffn(x_norm1)

        x_ff = self.dropout(x_ff)
        x_norm2 = self.norm2(x_ff + x_norm1)
        
        return x_norm2

In [8]:
encoder = TransformerEncoder(
    d_model= 512,
    d_ff=2048,
    num_heads=4
)

In [9]:
enc = EmbeddingWithPositionalEncoding(
    vocab_size=100,
    d_embed=1024, 
    d_model=512
)

encoder(
    enc(torch.tensor([1, 2, 4], device=dev).unsqueeze(0))
    )
encoder

TransformerEncoder(
  (att_layer): TransformerAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (q_proj): Linear(in_features=512, out_features=512, bias=True)
    (k_proj): Linear(in_features=512, out_features=512, bias=True)
    (v_proj): Linear(in_features=512, out_features=512, bias=True)
    (output_proj): Linear(in_features=512, out_features=512, bias=True)
  )
  (ffn): FeedForwardNetwork(
    (fc1): Linear(in_features=512, out_features=2048, bias=True)
    (fc2): Linear(in_features=2048, out_features=512, bias=True)
  )
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

## Transformer Decoder

only the final encoder output is used for all the decoder layers.

In [10]:
class TransformerDecoder(nn.Module):
    def __init__(self,
                 d_model: int,
                 num_heads: int,
                 d_ff: int,
                 dropout_p = 0.1):
        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.dropout_p = dropout_p

        self.att_layer1 = TransformerAttention(
            d_model=self.d_model,
            num_heads=self.num_heads,
            dropout_p=self.dropout_p
        )

        self.norm1 = nn.LayerNorm(self.d_model, device=dev)

        self.att_layer2 = TransformerAttention(
            d_model=self.d_model,
            num_heads=self.num_heads,
            dropout_p=self.dropout_p
        )

        self.norm2 = nn.LayerNorm(self.d_model, device=dev)

        self.ffn = FeedForwardNetwork(
            d_model=self.d_model,
            d_ff = self.d_ff
        )

        self.norm3 = nn.LayerNorm(self.d_model, device=dev)

        self.dropout = nn.Dropout(p=self.dropout_p)

    @staticmethod
    def create_causal_mask(seq_len: int) -> torch.Tensor:
        mask = torch.triu(torch.ones(seq_len, seq_len, device=dev), diagonal=1)
        mask = mask.masked_fill(mask == 1, value=float('-inf'))
        return mask

    def forward(self, x: torch.Tensor, 
                cross_input:torch.Tensor,
                padding_mask:torch.Tensor = None
                ):
        batch_size, seq_length, d_model = x.size()

        causal_mask = self.create_causal_mask(seq_len=seq_length)
        causal_mask = causal_mask.unsqueeze(0).unsqueeze(1) #unsqeeze the mast for self attention

        x_att1 = self.att_layer1( #self-attention
            seq=x,
            att_mask = causal_mask
        )

        x_att1 = self.dropout(x_att1)
        x_norm1 = self.norm1(x_att1 + x)

        x_att2 = self.att_layer2( #cross attention
            seq=x_norm1,
            key_value_states=cross_input,
            att_mask = padding_mask
        )

        x_att2 = self.dropout(x_att2)
        x_norm2 = self.norm2(x_att2)

        x_ff = self.ffn(x_norm2)

        x_ff = self.dropout(x_ff)
        x_norm3 = self.norm3(x_ff)
        
        return x_norm3


In [11]:
enc = EmbeddingWithPositionalEncoding(
    vocab_size=100,
    d_embed=1024, 
    d_model=512
)

decoder = TransformerDecoder(
    d_model=512,
    num_heads=4,
    d_ff=2048
)

x = encoder(
    enc(torch.tensor([1, 2, 4], device=dev).unsqueeze(0))
    )

decoder.forward(enc(torch.tensor([1, 2, 4], device=dev).unsqueeze(0)), x)


tensor([[[ 0.1115, -0.3260, -0.2035,  ...,  0.3012,  0.1688, -0.1281],
         [ 0.0558,  0.1714, -0.9992,  ...,  1.5448,  0.4007, -1.6595],
         [-0.3475,  1.1575, -0.9196,  ...,  0.7762, -0.4351, -2.1617]]],
       grad_fn=<NativeLayerNormBackward0>)

## Transformer Encoder and Decoder 

In [12]:
class TransformerEncoderDecoder(nn.Module):
    def __init__(self,
                 N_enc: int,
                 N_dec: int, 
                 d_model:int,
                 num_heads: int, 
                 d_ff: int,
                 dropout_p = 0.1
                 ):
        
        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.dropout_p = dropout_p

        self.encoder_stack = nn.ModuleList([
            TransformerEncoder(
                d_model=self.d_model,
                num_heads=self.num_heads,
                d_ff=self.d_ff,
                dropout_p=self.dropout_p
            ) for _ in range(N_enc)
        ])

        self.decoder_stack = nn.ModuleList([
            TransformerDecoder(
                d_model=self.d_model,
                num_heads=self.num_heads,
                d_ff=self.d_ff,
                dropout_p=self.dropout_p
            )
        ])

    def forward(self, x: torch.Tensor, y:torch.Tensor) -> torch.Tensor:
        #pass through the encoder stack
        encoder_output = x
        for encoder in self.encoder_stack:
            encoder_output = encoder(encoder_output)

        #pass through the decoder stack
        #uses only the final encoder input
        decoder_output = y
        for decoder in self.decoder_stack:
            decoder_output = decoder(decoder_output, cross_input=encoder_output)

        return decoder_output

In [13]:
enc = EmbeddingWithPositionalEncoding(
    vocab_size=100,
    d_embed=1024, 
    d_model=512
)

tf = TransformerEncoderDecoder(
    N_enc = 6,
    N_dec=6,
    d_model=512,
    num_heads=4,
    d_ff=2048
)

tf(enc(
    torch.tensor([1, 2, 3], device=dev).unsqueeze(0)
    ), enc(
    torch.tensor([1, 2, 3], device=dev).unsqueeze(0)
    ))

tensor([[[-0.9166, -1.1369,  0.7707,  ..., -0.5880, -0.4792, -0.8575],
         [-0.6653, -1.5406, -0.7790,  ..., -1.5379,  0.0693, -0.4902],
         [-0.2039, -2.0182,  0.8717,  ..., -1.4624, -0.3908, -0.1823]]],
       grad_fn=<NativeLayerNormBackward0>)

## Full Transformer

In [14]:
class Transformer(nn.Module):
    def __init__(self,
                 N_enc: int,
                 N_dec: int,
                 vocab_size:int, 
                 d_embed: int,
                 d_model: int,
                 num_heads: int,
                 d_ff: int,
                 d_tgt_vocab: int,
                 dropout_p = 0.1):
        super().__init__()

        self.N_enc = N_enc
        self.N_dec = N_dec
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.d_embed = d_embed
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.dropout_p = dropout_p
        self.d_tgt_vocab = d_tgt_vocab

        self.src_embedder = EmbeddingWithPositionalEncoding(
            vocab_size=self.vocab_size,
            d_embed=self.d_embed,
            d_model=self.d_model,
            dropout_p=self.dropout_p
        )

        self.tgt_embedder = EmbeddingWithPositionalEncoding(
            vocab_size=self.vocab_size,
            d_embed=self.d_embed,
            d_model=self.d_model,
            dropout_p=self.dropout_p
        )

        self.encoder_decoder_stack = TransformerEncoderDecoder(
            N_enc=self.N_enc,
            N_dec=self.N_dec,
            d_model=self.d_model,
            num_heads=self.num_heads,
            d_ff=self.d_ff,
            dropout_p=self.dropout_p
        )

        self.output_proj = nn.Linear(
            in_features=self.d_model,
            out_features=self.d_tgt_vocab,
            device=dev
        )

        self.softmax = nn.LogSoftmax(dim=-1)

    @staticmethod
    def shift_target_right(tgt_tokens: torch.Tensor) -> torch.Tensor:
        batch_size, seq_len = tgt_tokens.size() # no d_model since, no Embedding done
        zer = torch.zeros(
            size=(batch_size, 1),
            device=dev,
            dtype=torch.long
        )
        return torch.concat([
            zer, 
            tgt_tokens[:, :-1]], 
            dim=1).to(dev)

    def forward(self, 
                src_tokens:torch.Tensor, 
                tgt_tokens:torch.Tensor
                ) -> torch.Tensor:
        
        tgt_tokens = self.shift_target_right(tgt_tokens) 
        # shifting is needed to prevent information leakage. 
        # it allows parallel traning in spite of hiding the token 
        # to be predicted.
        inp_embed = self.src_embedder(src_tokens)
        tgt_embed = self.tgt_embedder(tgt_tokens)

        enc_dec_out = self.encoder_decoder_stack.forward(inp_embed, tgt_embed)
        
        out = self.output_proj(enc_dec_out)
        log_probs = self.softmax(out)
        return log_probs


In [15]:
tf = Transformer(
    N_enc = 6,
    N_dec=6,
    d_model=512,
    num_heads=4,
    d_ff=2048,
    vocab_size=100,
    d_embed=1024,
    d_tgt_vocab=100
)

In [16]:
total_params = sum(p.numel() for p in tf.parameters())
print(f"Total parameters: {total_params:,}")

Total parameters: 24,426,084


In [17]:
tf(
    src_tokens=torch.tensor([1., 2., 3.], device=dev, dtype=torch.long).unsqueeze(0),
    tgt_tokens=torch.tensor([1., 2., 3.], device=dev, dtype=torch.long).unsqueeze(0),
).shape

torch.Size([1, 3, 100])